# HarvestStat Data Profiling - Uganda

In [1]:
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32736' # Uganda
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/UG_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Uganda',
    'product': ['R011','R012','R013','R014','R015','R016','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
print(df.groupby(['status','collection_status','data_usage_policy']).size().reset_index(name='count'))
# ----------------------------------------------- #

# The current data consists of 4 years of records (1990-1991 and 2007-2008)
# We temporarily filter out the data before 2007
df = df[df['fnid'].apply(lambda x: x[:8]) == 'UG2007A2'].reset_index(drop=True)

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1990	 | UG1990A1 | 4	| UG1990A2	| 34	| nan	| 0	|
| 1991	 | UG1991A1 | 4	| UG1991A2	| 38	| nan	| 0	|
| 1994	 | UG1994A1 | 4	| UG1994A2	| 39	| nan	| 0	|
| 1997	 | UG1997A1 | 4	| UG1997A2	| 45	| nan	| 0	|
| 2001	 | UG2001A1 | 4	| UG2001A2	| 56	| nan	| 0	|
| 2005	 | UG2005A1 | 4	| UG2005A2	| 69	| nan	| 0	|
| 2007	 | UG2007A1 | 4	| UG2007A2	| 80	| nan	| 0	|
| 2010	 | UG2010A1 | 4	| UG2010A2	| 112	| nan	| 0	|
| 2014	 | UG2014A1 | 4	| UG2014A2	| 112	| nan	| 0	|
| 2021	 | UG2021A1 | 4	| UG2021A2	| 135	| nan	| 0	|
----------------------------------------------- #
          status collection_status data_usage_policy  count
0      Collected         Published            Public   6799
1  Not Collected         Published            Public   2725


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# Change season names
df['season_name'] = df['season_name'].replace({
    'First harvest': 'First',
    'Second harvest': 'Second',
    'Annual harvest': 'Annual'
})
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([UG_Admin2_1990, UG_Admin2_2007], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 7,672
Removed 2,248 "Missing Value" points
0/1,918 "Area Harvested" points are retained.
1,889/1,918 "Area Planted" points are retained.
1,782/1,918 "Quantity Produced" points are retained.
1,753/1,918 "Yield" points are retained.
Current data points: 5,424

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 2008 - 2009
16 grain types are found: Banana (unspecified), Beans (mixed), Cassava, Cooking Banana (unspecified), Cowpeas (Mixed), Field Peas, Groundnuts (In Shell), Maize Grain (White), Millet (Finger), Pigeon Peas, Potato (Irish), Rice (Paddy), Sesame Seed, Sorghum, Soybean (unspecified), Sweet Potatoes
3 seasons are found: First (01-01), Second (07-01), Annual (01-01)
1 crop production system are found: none
Data sources include:
[1] Bureau of Statistics, Uganda --- Census of Agriculture and Livestock, Uganda
Administrative-1 fnids: 0
Administrat

- Uganda crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-uganda.png?itok=ka9niXoM)

- FDW data consists of `UG1990A2` and `UG2007A2`.

| Year  | Admin-1  | # units  | Admin-2  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   |
| 1990  | UG1990A1 | 4        | **`UG1990A2`** | 34      |
| 1991  | UG1991A1 | 4        | UG1991A2 | 38      |
| 1994  | UG1994A1 | 4        | UG1994A2 | 39      |
| 1997  | UG1997A1 | 4        | UG1997A2 | 45      |
| 2001  | UG2001A1 | 4        | UG2001A2 | 56      |
| 2005  | UG2005A1 | 4        | UG2005A2 | 69      |
| 2007  | UG2007A1 | 4        | **`UG2007A2`** | 80      |
| 2010  | UG2010A1 | 4        | UG2010A2 | 112     |
| 2014  | UG2014A1 | 4        | UG2014A2 | 112     |
| 2021  | UG2021A1 | 4        | UG2021A2 | 135     |

- Comparison between admin boundaries.

<img src="../figures/UG_admin_shapes.png"  width="60%">

- **FDW data consists of only 5 years of records**.
- **`UG2007A2`** is used to represent the current admin-level 2 crop data.
- Uganda has three crop seasons: `Annual harvest`,`First harvest` and `Second harvest`.
- Uganda has no crop production system.
- Uganda has no population group.

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = UG_Admin2_2007.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area_new, prod_new = FDW_PD_Compiling(df, shape_used)
# ----------------------------------------------- #

In [5]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {
    'Banana (unspecified)': 'Banana',
    'Beans (mixed)': 'Beans (mixed)',
    'Cassava': 'Cassava',
    'Cooking Banana (unspecified)': 'Cooking Banana',
    'Cowpeas (Mixed)': 'Cowpeas',
    'Field Peas': 'Field Peas',
    'Groundnuts (In Shell)': 'Groundnuts (In Shell)',
    'Maize Grain (White)': 'Maize',
    'Millet (Finger)': 'Millet',
    'Pigeon Peas': 'Pigeon Peas',
    'Potato (Irish)': 'Potato',
    'Rice (Paddy)': 'Rice',
    'Sesame Seed': 'Sesame Seed',
    'Sorghum': 'Sorghum',
    'Soybean (unspecified)': 'Soybean',
    'Sweet Potatoes': 'Sweet Potatoes',
 }
assert set(stack['product'].unique()) == set(product_category_custom.keys())
stack['product'] = stack['product'].replace(product_category_custom)
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Uganda']
stack, _ = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio=None)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_UG.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')

All [season_name, product, crop_production_system] are in the external crop calendar.
../data/crop/adm_crop_production_UG.csv is saved.


# Visualization of production data

In [6]:
# Bar chart of national crop production
country_iso, country_name = 'UG', 'Uganda'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Sorghum','Millet','Maize','Rice','Fonio']
for season_name in ['Annual','First','Second']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

../figures/UG_bar_natgrainprod_Annual.png is saved.


../figures/UG_bar_natgrainprod_First.png is saved.


../figures/UG_bar_natgrainprod_Second.png is saved.


In [7]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'UG', 'Uganda'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','First'],
    ['Maize','Second'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

../figures/UG_line_pay_Maize_First.png is saved.


../figures/UG_line_pay_Maize_Second.png is saved.
